# Station-keeping Sequence

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

from ostk.core.filesystem import Directory

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics import Environment
from ostk.physics.coordinate import Frame
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth, Moon, Sun
from ostk.physics.time import DateTime, Duration, Instant, Scale, Time, Interval
from ostk.physics.units import Mass, Length

from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.event_condition import RealCondition
from ostk.astrodynamics.trajectory.orbit.models.brouwerLyddaneMean import (
    BrouwerLyddaneMeanShort
)
from ostk.astrodynamics.trajectory import StateBuilder
from ostk.astrodynamics.trajectory.state import CoordinatesBroker, CoordinatesSubset
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianPosition, CartesianVelocity
from ostk.astrodynamics.trajectory import Orbit, State, Sequence
from ostk.astrodynamics import Dynamics
from ostk.astrodynamics.flight.system import SatelliteSystem, PropulsionSystem
from ostk.astrodynamics.dynamics.thruster import ConstantThrust
from ostk.astrodynamics.trajectory.orbit.models import Tabulated

## User inputs

Setup environment

In [ ]:
earth = Earth.from_models(
    EarthGravitationalModel(EarthGravitationalModel.Type.EGM96, Directory.undefined(), 20, 20),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.Exponential),
)

environment = Environment(Instant.J2000(), [earth])

Initial state

In [ ]:
instant = Instant.date_time(DateTime(2023,1,1), Scale.UTC)
initial_state = Orbit.sun_synchronous(instant, Length.kilometers(505.0), Time.midnight(), Earth.default()).get_state_at(instant)

In [ ]:
dry_mass = Mass.kilograms(100.0)
wet_mass = Mass.kilograms(10.0)

propulsion_system = PropulsionSystem(thrust_si_unit=1e-1, specific_impulse_si_unit=3000.0)

## Setup Dynamics, initial state and Satellite System

In [ ]:
satellite_geometry = Composite(Cuboid(
    Point(0.0, 0.0, 0.0), np.eye(3).tolist(), [1.0, 0.0, 0.0]
))

satellite_system = SatelliteSystem(dry_mass, satellite_geometry, np.eye(3), 500.0, 2.2, propulsion_system)

In [ ]:
state_builder = StateBuilder(
    frame=Frame.GCRF(),
    coordinates_subsets=[
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
        CoordinatesSubset.surface_area(),
        CoordinatesSubset.drag_coefficient(),
    ],
)

coordinates = [
    *initial_state.get_coordinates().tolist(),
    dry_mass.in_kilograms() + wet_mass.in_kilograms(),
    satellite_system.get_cross_sectional_surface_area(),
    satellite_system.get_drag_coefficient()
]

state = state_builder.build(initial_state.get_instant(), coordinates)

In [ ]:
dynamics = Dynamics.from_environment(environment)
numerical_solver = NumericalSolver.default_conditional()

### Sequence

In [ ]:
sequence = Sequence(
    repetition_count=2,
    numerical_solver=numerical_solver,
    dynamics=dynamics,
    maximum_propagation_duration=Duration.days(30.0),
    verbosity=5
)

evaluator = lambda state: BrouwerLyddaneMeanShort.cartesian((state.get_position(), state.get_velocity()), EarthGravitationalModel.EGM2008.gravitational_parameter).get_semi_major_axis().in_meters()

# coast till under 495.0
coast_condition = RealCondition(
    name="Mean SMA crossing - coast",
    criterion=RealCondition.Criterion.StrictlyNegative,
    evaluator=evaluator,
    target=EarthGravitationalModel.EGM2008.equatorial_radius.in_meters() + 495.0e3,
)
sequence.add_coast_segment(coast_condition)

# burn till at 500.0
thrust_condition = RealCondition(
    name="Mean SMA crossing - thrust",
    criterion=RealCondition.Criterion.StrictlyPositive,
    evaluator=evaluator,
    target=EarthGravitationalModel.EGM2008.equatorial_radius.in_meters() + 500.0e3,
)
sequence.add_maneuver_segment(
    event_condition=thrust_condition,
    thruster_dynamics=ConstantThrust.intrack(satellite_system),
)

In [ ]:
sol = sequence.solve(state)
states = sol.get_states()

In [ ]:
model = Tabulated(states, 0, Tabulated.InterpolationType.BarycentricRational)

In [ ]:
orbit = Orbit(model, environment.access_celestial_object_with_name("Earth"))

In [ ]:
interval = Interval.closed(states[0].get_instant(), states[-1].get_instant())
interpolated_states = orbit.get_states_at(interval.generate_grid(Duration.seconds(20.0)))

In [ ]:
data = []
for state in interpolated_states:
    blmshort = BrouwerLyddaneMeanShort.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "altitude": float(blmshort.get_semi_major_axis().in_kilometers() - earth.get_equatorial_radius().in_kilometers()),
            "mass": float(state.get_coordinates()[6]),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )
    

In [ ]:
df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="altitude", color="mass")
for segment_solution in sol.segment_solutions:
    time = segment_solution.states[-1].get_instant().get_date_time(Scale.UTC)
    fig.add_vline(
        x=time.isoformat()
    )
fig.show()

In [ ]:
fig2 = px.line(df, x="time", y="mass")
for segment_solution in sol.segment_solutions:
    time = segment_solution.states[-1].get_instant().get_date_time(Scale.UTC)
    fig2.add_vline(
        x=time.isoformat()
    )
fig2.show()